In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
steam_df = pd.read_csv('./steam-store-games/steam.csv', usecols = [0, 1, 8, 9], low_memory=False)
steam_df = steam_df.rename(columns={'name': 'game_name'}).rename(columns={'appid': 'game_id'})
steam_df.drop_duplicates(inplace=True)
steam_df.to_csv('./dataset/steam.csv', index=False)
steam_df

,game_id,game_name,categories,genres
0,10,Counter-Strike,Multi-player;Online Multi-Player;Local Multi-P...,Action
1,20,Team Fortress Classic,Multi-player;Online Multi-Player;Local Multi-P...,Action
2,30,Day of Defeat,Multi-player;Valve Anti-Cheat enabled,Action
3,40,Deathmatch Classic,Multi-player;Online Multi-Player;Local Multi-P...,Action
4,50,Half-Life: Opposing Force,Single-player;Multi-player;Valve Anti-Cheat en...,Action
...,...,...,...,...
27070,1065230,Room of Pandora,Single-player;Steam Achievements,Adventure;Casual;Indie
27071,1065570,Cyber Gun,Single-player,Action;Adventure;Indie
27072,1065650,Super Star Blast,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie
27073,1066700,New Yankee 7: Deer Hunters,Single-player;Steam Cloud,Adventure;Casual;Indie


In [ ]:
file_path = Path("archive/")
reviews_df = pd.concat([pd.read_csv(i, usecols=[0,1,2]) for  i in file_path.glob('**/*.csv')])
reviews_df = reviews_df.rename(columns={'steamid': 'user_id'}).rename(columns={'appid': 'game_id'})
reviews_df['voted_up'] = reviews_df['voted_up'].astype(int)
reviews_df

,user_id,game_id,voted_up
0,76561198107294407,10,1
1,76561198011733201,10,1
2,76561198168961276,10,1
3,76561198957877160,10,1
4,76561199050314447,10,1
...,...,...,...
623037,76561198044371296,535520,1
623038,76561198027331588,535690,1
623039,76561198072234306,535690,1
623040,76561198011544203,535690,0


In [ ]:
reviews_df = (
    reviews_df
    .join(
        steam_df[['game_id', 'game_name', 'genres']]
        .set_index('game_id'), 
        on='game_id', 
        how='inner',
    )
)
reviews_df.drop_duplicates(inplace=True)
reviews_df.to_csv('./dataset/reviews.csv', index=False)
reviews_df

,user_id,game_id,voted_up,game_name,genres
0,76561198107294407,10,1,Counter-Strike,Action
1,76561198011733201,10,1,Counter-Strike,Action
2,76561198168961276,10,1,Counter-Strike,Action
3,76561198957877160,10,1,Counter-Strike,Action
4,76561199050314447,10,1,Counter-Strike,Action
...,...,...,...,...,...
623037,76561198044371296,535520,1,Nidhogg 2,Action;Casual;Indie
623038,76561198027331588,535690,1,Crash Force®,Action;Indie;RPG
623039,76561198072234306,535690,1,Crash Force®,Action;Indie;RPG
623040,76561198011544203,535690,0,Crash Force®,Action;Indie;RPG


In [ ]:
len(reviews_df['user_id'].unique()), len(reviews_df['game_id'].unique())

(6976390, 8183)

In [ ]:
reviews_df['voted_up'].value_counts(normalize=True)

1    0.869593
0    0.130407
Name: voted_up, dtype: float64

In [ ]:
processed_df = pd.concat([reviews_df[reviews_df['voted_up'] == 1].sample(frac=0.4), reviews_df[reviews_df['voted_up'] == 0]])
processed_df

,user_id,game_id,voted_up,game_name,genres
1629943,76561198329805440,311210,1,Call of Duty®: Black Ops III,Action;Adventure
843039,76561198038418440,364360,1,Total War: WARHAMMER,Action;Strategy
911900,76561197969749884,221910,1,The Stanley Parable,Adventure;Indie
671460,76561198061903067,431960,1,Wallpaper Engine,Animation & Modeling;Design & Illustration;Uti...
409963,76561198054463336,640820,1,Pathfinder: Kingmaker,RPG
...,...,...,...,...,...
623020,76561198060219692,535520,0,Nidhogg 2,Action;Casual;Indie
623025,76561197972319712,535520,0,Nidhogg 2,Action;Casual;Indie
623027,76561198005510863,535520,0,Nidhogg 2,Action;Casual;Indie
623036,76561198256911090,535520,0,Nidhogg 2,Action;Casual;Indie


In [ ]:
processed_df['voted_up'].value_counts(normalize=True)

1    0.727321
0    0.272679
Name: voted_up, dtype: float64

In [ ]:
nuniq_reviews = processed_df.groupby('user_id').size()
nuniq_reviews.mean()

1.7623046500147523

In [ ]:
nuniq_over4 = nuniq_reviews[nuniq_reviews >= 4]
processed_df = processed_df[processed_df['user_id'].isin(nuniq_over4.index)]
processed_df

,user_id,game_id,voted_up,game_name,genres
843039,76561198038418440,364360,1,Total War: WARHAMMER,Action;Strategy
911900,76561197969749884,221910,1,The Stanley Parable,Adventure;Indie
110947,76561198067364909,280,1,Half-Life: Source,Action
757490,76561198066909363,583470,1,The End Is Nigh,Action;Adventure;Indie
195986,76561197971370772,346110,1,ARK: Survival Evolved,Action;Adventure;Indie;Massively Multiplayer;RPG
...,...,...,...,...,...
622980,76561198148817144,535520,0,Nidhogg 2,Action;Casual;Indie
622998,76561198034691818,535520,0,Nidhogg 2,Action;Casual;Indie
623020,76561198060219692,535520,0,Nidhogg 2,Action;Casual;Indie
623025,76561197972319712,535520,0,Nidhogg 2,Action;Casual;Indie


In [ ]:
processed_df['voted_up'].value_counts(normalize=True)

1    0.653608
0    0.346392
Name: voted_up, dtype: float64

In [ ]:
processed_df.groupby('user_id').size().mean()

7.087262778840215

In [ ]:
len(processed_df['user_id'].unique()), len(processed_df['game_id'].unique())

(335183, 8010)

In [ ]:
processed_df.to_csv('./dataset/processed_reviews.csv', index=False)

In [ ]:
genre_explode_df = (
    processed_df
    .assign(
        genres=lambda df: df['genres'].str.split(';'),
    )
    .explode('genres')
    .rename(columns={'genres': 'genre'})
    .query('~genre.isna()')
)
genre_explode_df

,user_id,game_id,voted_up,game_name,genre
843039,76561198038418440,364360,1,Total War: WARHAMMER,Action
843039,76561198038418440,364360,1,Total War: WARHAMMER,Strategy
911900,76561197969749884,221910,1,The Stanley Parable,Adventure
911900,76561197969749884,221910,1,The Stanley Parable,Indie
110947,76561198067364909,280,1,Half-Life: Source,Action
...,...,...,...,...,...
623025,76561197972319712,535520,0,Nidhogg 2,Casual
623025,76561197972319712,535520,0,Nidhogg 2,Indie
623040,76561198011544203,535690,0,Crash Force®,Action
623040,76561198011544203,535690,0,Crash Force®,Indie


In [ ]:
genre_df = pd.DataFrame([
    {'genre_id': i, 'genre_name': genre}
    for i, genre in enumerate(genre_explode_df['genre'].sort_values().unique())
])
genre_df
genre_df.to_csv('./dataset/genres.csv', index=False)

In [ ]:
processed_genre_df = (
    genre_explode_df
    .join(
        genre_df.set_index('genre_name'), 
        on='genre',
    )
)
processed_genre_df

,user_id,game_id,voted_up,game_name,genre,genre_id
843039,76561198038418440,364360,1,Total War: WARHAMMER,Action,0
843039,76561198038418440,364360,1,Total War: WARHAMMER,Strategy,21
911900,76561197969749884,221910,1,The Stanley Parable,Adventure,1
911900,76561197969749884,221910,1,The Stanley Parable,Indie,11
110947,76561198067364909,280,1,Half-Life: Source,Action,0
...,...,...,...,...,...,...
623025,76561197972319712,535520,0,Nidhogg 2,Casual,4
623025,76561197972319712,535520,0,Nidhogg 2,Indie,11
623040,76561198011544203,535690,0,Crash Force®,Action,0
623040,76561198011544203,535690,0,Crash Force®,Indie,11


In [ ]:
len(processed_genre_df['user_id'].unique()), len(processed_genre_df['genre_id'].unique())

(335183, 26)

In [ ]:
processed_genre_df['voted_up'].value_counts(normalize=True)

1    0.636917
0    0.363083
Name: voted_up, dtype: float64

In [ ]:
processed_genre_df.to_csv('./dataset/processed_reviews+genre.csv', index=False)